## **Pomocne moduly**

In [49]:
import itertools
import math
import numpy as np
import UQpy
from scipy.special import comb
from UQpy.distributions import Uniform, JointIndependent
from UQpy.surrogates.polynomial_chaos import TotalDegreeBasis, LeastSquareRegression
from UQpy.sensitivity import PceSensitivity
from UQpy.sampling import MonteCarloSampling
from sklearn.metrics import mean_squared_error, r2_score
import time  
import scipy as sp

Generování multiindexových množin

In [52]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb  6 12:57:36 2023

@author: D. Loukrezis
"""

import itertools
import math
import numpy as np
from scipy.special import comb


def setsize(N, w):
    """Vypočítá počet PCE polynomů pro N dimenzí a maximální stupeň w pomocí kombinatoriky"""
    return int(comb(N+w-1, N-1))    


def td_set_recursive(N, w, rows):
    """Rekurzivní funkce pro generování multiindexů totálního stupně"""
    if N == 1:  # Základní případ - 1D
        subset = w*np.ones([rows, 1])
    else:
        if w == 0:  # Nulový stupeň - všechny nuly
            subset = np.zeros([rows, N])
        elif w == 1:  # Stupeň 1 - jednotková matice
            subset = np.eye(N)
        else:
            # Inicializace podmatice
            subset = np.empty([rows, N])
            
            # Počáteční řádek podmatice
            row_start = 0
            
            # Iterace přes polynomiální stupně
            for k in range(0, w+1):
                
                # Počet řádků podmatice
                sub_rows = setsize(N-1, w-k)
                
                # Aktualizace koncového řádku
                row_end = row_start + sub_rows - 1
                
                # První sloupec - nastaví hodnotu k
                subset[row_start:row_end+1, 0] = k*np.ones(sub_rows)
                
                # Rekurzivní volání pro zbylé dimenze
                subset[row_start:row_end+1, 1:] = td_set_recursive(N-1, w-k, 
                                                              sub_rows)
                                                                     
                # Aktualizace indexu řádku
                row_start = row_end + 1
    
    return subset


def td_multiindex_set(N, w):
    """Generuje multiindex set totálního stupně pro N parametrů a max stupeň w"""
    
    # Velikost množiny multiindexů
    td_size = int(comb(N+w, N))
    
    # Inicializace výsledné množiny
    midx_set = np.empty([td_size, N])
    
    # Počáteční řádek
    row_start = 0
    
    # Iterace přes polynomiální stupně
    for i in range(0, w+1):
        
        # Výpočet počtu řádků pro daný stupeň
        rows = setsize(N, i)
        
        # Aktualizace koncového řádku
        row_end = rows + row_start - 1
        
        # Rekurzivní generování části množiny
        midx_set[row_start:row_end+1, :] = td_set_recursive(N, i, rows)
        
        # Aktualizace počátečního řádku
        row_start = row_end + 1
        
    return midx_set.astype(int)  # Převod na celá čísla


def tp_multiindex_set(N, w):
    """Generuje tenzorový součin multiindex set pro N parametrů a max stupeň w"""
    orders = np.arange(0, w+1, 1).tolist()  # Seznam stupňů 0 až w
    if N == 1:  # Speciální případ pro 1D
        midx_set = np.array(list(map(lambda el:[el], orders)))
    else:
        # Generování všech kombinací (kartézský součin)
        midx = list(itertools.product(orders, repeat=N))
        midx = [list(elem) for elem in midx]
        # Výpočet součtů pro každý multiindex
        midx_sums = [int(math.fsum(midx[i])) for i in range(len(midx))]
        # Seřazení podle součtu (totální stupeň)
        midx_sorted = sorted(range(len(midx_sums)), 
                             key=lambda k: midx_sums[k])
        # Sestavení výsledného pole
        midx_set = np.array([midx[midx_sorted[i]] for i in range(len(midx))])   
    return midx_set.astype(int)  # Převod na celá čísla

In [54]:
import numpy as np

def is_admissible(index, index_set):
    """
    Zjistí, zda je daný multi-index přípustný
    (všechny jeho zpětné sousedy jsou v index_set)
    
    Parametry:
    index : np.array - testovaný multi-index
    index_set : list - seznam multi-indexů v aktuální bázi
    """
    back_neighbors = backward_neighbors(index)  # Získá zpětné sousedy
    
    for ind_b in back_neighbors:
        # Oprava: porovnání polí po prvcích
        if not any(np.array_equal(ind_b, x) for x in index_set):
            return False
    return True

def admissible_neighbors(index, index_set):
    """
    Najde všechny přípustné sousedy daného indexu
    
    Parametry:
    index : np.array - výchozí multi-index
    index_set : list - seznam povolených indexů
    """
    for_neighbors = forward_neighbors(index)  # Získá dopředné sousedy
    
    # Filtruje jen přípustné sousedy
    for_truefalse = [is_admissible(fn, index_set) for fn in for_neighbors]
    adm_neighbors = np.array(for_neighbors)[for_truefalse].tolist()
    
    return adm_neighbors


def forward_neighbors(index):
    """Vrátí dopředné sousedy - každou dimenzi zvýší o 1 (např. (2,1) → (3,1), (2,2))"""
    N = len(index)  # Počet dimenzí multiindexu
    for_neighbors = []
    for i in range(N):  # Pro každou dimenzi
        index_tmp = index[:]  # Kopie původního indexu
        index_tmp[i] = index_tmp[i] + 1  # Inkrementace aktuální dimenze
        for_neighbors.append(index_tmp)  # Přidání do seznamu sousedů
    return for_neighbors


def backward_neighbors(index):
    """Vrátí zpětné sousedy - každou dimenzi sníží o 1 (pokud možno, např. (2,2) → (1,2), (2,1))"""
    N = len(index)  # Počet dimenzí multiindexu
    back_neighbors = []
    for i in range(N):  # Pro každou dimenzi
        index_tmp = index[:]  # Kopie původního indexu
        if index_tmp[i] > 0:  # Pokud lze snížit (nenulová hodnota)
            index_tmp[i] = index_tmp[i] - 1  # Dekrementace aktuální dimenze
            back_neighbors.append(index_tmp)  # Přidání do seznamu sousedů
    return back_neighbors



In [56]:
!pip install UQpy
!pip install --upgrade UQpy
!pip install torch

## **PCE** = UQPY

In [113]:
# -*- coding: utf-8 -*-
import openturns as ot
import numpy as np
import scipy as sp

class PolynomialChaosExpansion():
       #Generuje multiindexy pro polynomy 1. stupn
   
    def __init__(self, pdf, exp_design_in, exp_design_out):
        
        # případ bez zadaného PDF: identifikuj rozdělení z dat
        if pdf == None:
            chaos_algo_data = ot.FunctionalChaosAlgorithm(exp_design_in, 
                                                          exp_design_out)
            chaos_algo_data.run()
            self.pdf = chaos_algo_data.getDistribution()
        else:    
            self.pdf = pdf
        self.num_inputs = len(pdf.marginals)                  
        self.num_outputs = exp_design_out.shape[1]
        self.num_samples = exp_design_out.shape[0]
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out
        
        max_degree=2
        # získání enumerátoru pro single a multi-indexy
        self.enumerate_function = TotalDegreeBasis(pdf, max_degree).multi_index_set
       
        
        ## přiřazení správných polynomů podle vstupního rozdělení NENí POTREBA UQPY TO DELA AUTOMATICKY
        #self.polynomial_collection = [
                #  ot.StandardDistributionPolynomialFactory(
                     # self.pdf.getMarginal(i))
               #   for i in range(self.num_inputs)]
        
       
        # vytvoření obecné báze pro konstrukci PCE
        self.product_basis = TotalDegreeBasis(pdf, max_degree)
       #  print(f"Pocet bazi: { self.product_basis.polynomials_number}")
       #  print(f"Multi-index set: { self.product_basis.multi_index_set}")
        self.single_index_set = list(range(len(self.product_basis.multi_index_set)))  # [0, 1, 2, ...]
       #  print(f"Multi-index set: {self.single_index_set}")  #stejny vystup jako vopenturns
        
        # vytvoření transformační funkce - velmi důležité! - NENí POTREBA UQPY TO DELA AUTOMATICKY
        # self.transformation = ot.DistributionTransformation(
                                 #self.pdf, self.product_basis.getMeasure())
       
    #NENí POTREBA UQPY TO DELA AUTOMATICKY-  TotalDegreeBasis, PolynomialChaosExpansion 
     # automaticky generují a spravují multi-indexy
     # def set_multi_index_set(self, multi_index_set):
        #"""Nastaví množinu multi-indexů a přepočítá single-indexy"""
        #self.multi_index_set = multi_index_set
        #self.single_index_set = [self.enumerate_function.inverse(idx) 
                                 #for idx in self.multi_index_set]
    
    #  Nní potřeba explicitně nastavovat single_index_set, automaticky generují při vytvoření báze    
    #def set_single_index_set(self, single_index_set):
        #"""Nastaví množinu single-indexů a přepočítá multi-indexy"""
       # self.single_index_set = single_index_set
       # self.multi_index_set = [list(self.enumerate_function(idx)) 
                                     # for idx in self.single_index_set]
        
    #def construct_basis(self):
       # """Sestaví bázi polynomů na základě nastavených indexů"""
       # self.basis = self.product_basis.getSubBasis(self.single_index_set)
       # self.num_polynomials = self.basis.getSize()

    #Kontroluje, jestli nová data sedí .Zkontroluje, že počet výstupů  v nových datech je
    #stejný jako u původních dat. Pokud ne, vyhodí chybu.
    def set_exp_design(self, exp_design_in, exp_design_out):
        exp_design_out = np.array(exp_design_out) # Převod výstupních dat na numpy array 
    # Pokud jsou výstupní data 1D , převedna 2D tvar
        if len(exp_design_out.shape) == 1:
                exp_design_out = exp_design_out.reshape(-1, 1)
    # Pokud ještě není nastaven počet výstupů (první volání funkce), uložíme ho
        if self.num_outputs is None:
                self.num_outputs = exp_design_out.shape[1]
    # Pokud už počet výstupů známe, zkontrolujeme shodu s novými daty
        elif self.num_outputs != exp_design_out.shape[1]:
            raise ValueError('Výstupní dimenze nesouhlasí!')
        self.exp_design_inputs = np.array(exp_design_in) # Uložení vstupních dat
        self.exp_design_outputs = exp_design_out# Uložení výstupních dat
        self.num_samples = exp_design_out.shape[0] # Uložení počtu vzorků
        self.pce.fit(x=self.exp_design_inputs, y=self.exp_design_outputs) #Přetrénování modelu

     # NENí POTREBA UQPY TO DELA AUTOMATICKY- 
    #  vyhodnocuje polynomiální bázové funkce pro zadané vstupní body
    #def evaluate_basis(self, design_in):
       # """Vyhodnotí bázové funkce pro dané vstupní body"""
        # n_samples, n_inputs = np.shape(design_in)
        # if n_inputs != self.num_inputs:
           #  raise ValueError('Vstupní dimenze nesouhlasí!')
        # transformace vstupních dat
        # design_in_tf = np.array(self.transformation(design_in))
        # výpočet matice vyhodnocení báze
        #eval_matrix = np.array([self.basis[j](design_in_tf) 
                                #for j in range(self.num_polynomials)])
        # odstranění nadbytečné dimenze
       #  eval_matrix = np.squeeze(eval_matrix)
        # return eval_matrix.T

    # Vypočítá návrhovou matici"""
    def compute_design_matrix(self):
        self.design_matrix =  self.product_basis.evaluate_basis(self.exp_design_inputs)
        # print("\033[95m" + str(self.design_matrix) + "\033[0m")
    
    # koeficienty PCE    
    def compute_coefficients(self):
    # Nejprve vypočítá návrhovou matici 
        self.compute_design_matrix()
    # Metoda nejmenších čtverců pro výpočet koeficientů
    # np.linalg.lstsq řeší systém rovnic Xβ = y
    # Vrací: koeficienty, rezidua, hodnost matice, singulární hodnoty
    # rcond=None znamená automatický výběr tolerance pro hodnost
        self.coefficients, _, _, sv = np.linalg.lstsq(
            self.design_matrix, 
            self.exp_design_outputs, 
            rcond=None)
    # Výpočet čísla podmíněnosti 
    # kazuje citlivost řešení na změny vstupů
    # Vysoké hodnoty (>1e10) signalizují numerické problémy
        self.condition_number = sv[0]/sv[-1]
        return self.coefficients  # Vrácení vypočtených koeficientů
        
    #predikce výstupních hodnot  pomocí natrénovaného PC
    def predict(self, x: np.ndarray) -> np.ndarray: 
        eval_matrix = self.product_basis.evaluate_basis(x)  # Změněno z polynomial_basis na product_basis
        return eval_matrix @ self.coefficients

    def compute_mean(self):
        """Vypočítá střední hodnotu PCE modelu (první koeficient)"""
        if not hasattr(self, 'coefficients'):
            self.compute_coefficients()
        return self.coefficients[0, :] if self.coefficients.ndim > 1 else self.coefficients[0]

    def compute_variance(self):
        """Vypočítá rozptyl PCE modelu (součet čtverců ostatních koeficientů)"""
        if not hasattr(self, 'coefficients'):
            self.compute_coefficients()
        if self.coefficients.ndim > 1:  # Pro více výstupů
            return np.sum(self.coefficients[1:,:]**2, axis=0)
        else:  # Pro jeden výstup
            return np.sum(self.coefficients[1:]**2)

    def compute_stddev(self):
        """Vypočítá směrodatnou odchylku PCE modelu"""
        return np.sqrt(self.compute_variance())
    
        
    def compute_sobol_first(self):
        sobol_f = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        
        # Odstranění nulového multi-indexu
        midx_minus_0 = np.delete(self.multi_index_set, 0, axis=0)
        
        for i in range(self.num_inputs):
            # Najdi řádky kde jsou všechny ostatní indexy nulové
            midx_minus_i = np.delete(midx_minus_0, i, axis=1)
            row_sum = np.sum(midx_minus_i, axis=1)
            zero_rows = np.asarray(np.where(row_sum==0)).flatten() + 1
            
            # Částečný rozptyl pro i-tou proměnnou
            partial_variance = np.sum(np.square(self.coefficients[zero_rows]),
                                      axis=0)
            sobol_f[i,:] = partial_variance / variance
        return sobol_f
    
    def compute_sobol_total(self):
        sobol_t = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        mis = np.array(self.multi_index_set)
        
        for i in range(self.num_inputs):
            # Najdi řádky kde i-tý index není nula
            idx_column_i = mis[:,i] 
            non_zero_rows = np.asarray(np.where(idx_column_i!=0)).flatten()
            
            # Částečný rozptyl pro i-tou proměnnou
            partial_variance = np.sum(
                                np.square(self.coefficients[non_zero_rows]),
                                axis=0)
            sobol_t[i,:] = partial_variance / variance
        return sobol_t
    
    def compute_generalized_sobol_first(self):
        variance = self.compute_variance()
        sobol_f = self.compute_sobol_first()
        
        # Částečné rozptyly pro každou proměnnou
        partial_variances = sobol_f*variance
        
        # Agregovaný rozptyl (pro všechny výstupy dohromady)
        aggregated_variance = np.sum(variance)
        
        # Součet částečných rozptylů přes všechny výstupy
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        
        # Zobecněné indexy
        sobol_f_gen = aggregated_partial_variances / aggregated_variance
        return sobol_f_gen

    def compute_generalized_sobol_total(self):
        variance = self.compute_variance()
        sobol_t = self.compute_sobol_total()
        partial_variances = sobol_t*variance
        aggregated_variance = np.sum(variance)
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        sobol_t_gen = aggregated_partial_variances / aggregated_variance
        return sobol_t_gen
# Pridane zvlast

 



In [115]:
# Testuje samotné PCE
"""
import openturns as ot
import numpy as np
from UQpy.distributions import Normal, JointIndependent
from UQpy.surrogates.polynomial_chaos import TotalDegreeBasis

# 1. Vytvoření testovacích dat
dim = 3  # počet vstupních proměnných
size = 100  # velikost experimentálního designu

# Definice rozdělení vstupních proměnných
marginals = [Normal(loc=0.0, scale=1.0) for i in range(dim)]
distribution = JointIndependent(marginals=marginals)

# Generování vstupních vzorků
input_samples = distribution.rvs(nsamples=size)

# Definice testovací funkce (3 vstupy, 2 výstupy)
def test_function(x):
    y1 = x[0] + 2*x[1] + 0.5*x[2]**2
    y2 = 0.5*x[0]**2 + x[1] + 2*x[2]
    return [y1, y2]

# Výpočet výstupních hodnot
output_samples = np.array([test_function(x) for x in input_samples])

# 2. Vytvoření instance PCE
pce = PolynomialChaosExpansion(pdf=distribution, 
                             exp_design_in=np.array(input_samples), 
                             exp_design_out=output_samples)

# 3. Výpočet koeficientů
pce.compute_coefficients()

# 4. Testování funkcionality
print("\nTest Polynomial Chaos Expansion")
print("="*50)

# Základní informace
print(f"\nPočet vstupů: {pce.num_inputs}")
print(f"Počet výstupů: {pce.num_outputs}")
print(f"Počet vzorků: {pce.num_samples}")
print(f"Počet polynomů v bázi: {len(pce.product_basis.multi_index_set)}")
print(f"Podmíněné číslo: {pce.condition_number:.2f}")

# Predikce
test_input = np.array([[1.0, 0.5, -0.3], [0.2, -1.0, 0.7]])
prediction = pce.predict(test_input)
print("\nPredikce pro testovací vstupy:")
print("Vstupy:", test_input)
print("Výstupy:", prediction)

# Momenty (pro PCE)
print("\nStřední hodnoty (první koeficient):", pce.coefficients[0])
print("Rozptyly (součet čtverců koeficientů):", np.sum(pce.coefficients[1:]**2, axis=0))

# 5. Testování s jednovýstupovou funkcí
print("\nTestování s jednovýstupovou funkcí")
print("="*50)

def scalar_function(x):
    return x[0] + 2*x[1] + 0.5*x[2]**2

scalar_output = np.array([[scalar_function(x)] for x in input_samples])

pce_scalar = PolynomialChaosExpansion(pdf=distribution,
                                    exp_design_in=np.array(input_samples),
                                    exp_design_out=scalar_output)
pce_scalar.compute_coefficients()

print("\nKoeficienty pro skalární výstup:", pce_scalar.coefficients.flatten())

print("\nTestování dokončeno!")
    
"""

'\nimport openturns as ot\nimport numpy as np\nfrom UQpy.distributions import Normal, JointIndependent\nfrom UQpy.surrogates.polynomial_chaos import TotalDegreeBasis\n\n# 1. Vytvoření testovacích dat\ndim = 3  # počet vstupních proměnných\nsize = 100  # velikost experimentálního designu\n\n# Definice rozdělení vstupních proměnných\nmarginals = [Normal(loc=0.0, scale=1.0) for i in range(dim)]\ndistribution = JointIndependent(marginals=marginals)\n\n# Generování vstupních vzorků\ninput_samples = distribution.rvs(nsamples=size)\n\n# Definice testovací funkce (3 vstupy, 2 výstupy)\ndef test_function(x):\n    y1 = x[0] + 2*x[1] + 0.5*x[2]**2\n    y2 = 0.5*x[0]**2 + x[1] + 2*x[2]\n    return [y1, y2]\n\n# Výpočet výstupních hodnot\noutput_samples = np.array([test_function(x) for x in input_samples])\n\n# 2. Vytvoření instance PCE\npce = PolynomialChaosExpansion(pdf=distribution, \n                             exp_design_in=np.array(input_samples), \n                             exp_design_ou

## **SAPCE**

In [118]:
import numpy as np
from UQpy.distributions import Normal, JointIndependent
from UQpy.surrogates.polynomial_chaos import TotalDegreeBasis
from UQpy.surrogates.polynomial_chaos.regressions import LeastSquareRegression
#Generuje multiindexy 1. stupně - PCE
def generate_first_degree_indices(num_inputs):
    return np.eye(num_inputs, dtype=int).tolist()
    
class SensitivityAdaptivePCE:
    def __init__(self, pdf, exp_design_in, exp_design_out, num_inputs, max_partial_degree=10, tolerance=1e-3):
        self.pdf = pdf if pdf is not None else JointIndependent([Uniform(0, 1) for _ in range(exp_design_in.shape[1])])
        self.exp_design_in = exp_design_in
        self.exp_design_out = exp_design_out
        self.max_partial_degree = max_partial_degree
        self.tolerance = tolerance   # Ukládání do self zajišťuje, že hodnoty přežijí mimo __init__ a jsou dostupné v celé třídě.
        distribution= self.pdf

        #Pocet vstupu
        #num_inputs = self.pdf.getDimension() #Zjistí počet vstupních proměnných z pravděpodobnostního rozdělení 
        num_inputs = num_inputs

        #Vytvoří množinu multiindexů pro polynomy 1. stupně
        td1_set = globals()['generate_first_degree_indices'](num_inputs)
        #print("Multiindexy 1. stupně:", td1_set) 

        #Inicializace PCE
        #self.pce = PolynomialChaosExpansion(self.pdf, self.exp_design_in, self.exp_design_out) #Inicializuje PCE model, který bude aproximovat vztah mezi vstupy a výstupy
        #Nesmí být v kodu from UQpy.surrogates.polynomial_chaos import PolynomialChaosExpansion, kryje se 
        self.pce = PolynomialChaosExpansion(pdf=distribution, exp_design_in=np.array(self.exp_design_in), exp_design_out=self.exp_design_out)
        
        #Nastaví množinu multiindexů definovanou v  td1_set
        #self.pce.set_multi_index_set(td1_set) 
        self.multi_index_set = self.pce.product_basis.multi_index_set
        
        #Neni potreba
        #self.pce.construct_basis() #Sestaví ortogonální polynomiální bázi na základě
        
        #Spočítá koeficienty PCE metodou nejmenších čtverců
        self.pce.compute_coefficients() 

        #Začíná s nulovým multi-indexem
        self.active_multi_indices =  list([self.multi_index_set[0]])
        self.admissible_multi_indices =  list(self.multi_index_set[1:]) #Inicializace přípustných multi-indexůObsahuje všechny multi-indexy 1. stupně
        admissible_coefficients = self.pce.coefficients[1:] # Pro každý přípustný multi-index spočítá součet absolutních hodnot jeho koeficientů
        aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1) #absolutni hodnota, bez ohledu na zanemenko
        help_index = np.argmax(aggregated_admissible_coefficients)
       
        # Odstraní a uloží daný multi-index z admissible množiny
        max_admissible_multi_index = self.admissible_multi_indices.pop(help_index)
        # Přidá do aktivní množiny multi-indexů
        self.active_multi_indices.append(max_admissible_multi_index)
        # Odstraní odpovídající hodnotu z agregovaných koeficientů
        aggregated_admissible_coefficients = np.delete(aggregated_admissible_coefficients, help_index)

        print("Jsem tu v sapce")
           
    def construct_adaptive_basis(self, max_condition_number=1e2, termination_info=True):
        print("construct_adaptive_basis")

        while True:
            # T1: Kontroluje, zda číslo podmíněnosti aktuální matice překročilo limit
            if self.pce.condition_number > max_condition_number:
                if termination_info:
                    print("Adaptive basis construction terminated:" 
                        + " design matrix not sufficiently well-conditioned.")
                break
            
            # Hledání nových přípustných multi-indexů
            new_admissible_multi_indices = admissible_neighbors(
                                            self.active_multi_indices[-1],
                                            self.active_multi_indices)
            #print("new_admissible_multi_indices")
            #print("\033[95m" + str(new_admissible_multi_indices) + "\033[0m")
            # T2: Zajišťuje, že všechny nové přípustné multi-indexy splňují podmínku maximálního parciálního stupně, Stupně polynomu
    
            for idx, adm_multi_Indcs in reversed(list(enumerate(new_admissible_multi_indices))):
                for adm_multi_Indx in adm_multi_Indcs:
                    if adm_multi_Indx > self.max_partial_degree:
                        new_admissible_multi_indices.pop(idx)
                            
            #Kontroluje, zda číslo podmíněnosti aktuální matice překročilo limit
            #if [self.max_partial_degree]*self.pce.num_inputs in self.active_multi_indices: 
            target_index = np.array([self.max_partial_degree] * self.pce.num_inputs)
            index_found = any(np.array_equal(target_index, idx) for idx in self.active_multi_indices)
            if index_found: 
                if len(new_admissible_multi_indices) == 0:
                    if termination_info:
                        print("Adaptive basis construction terminated:" 
                        + " maximum partial degree reached.")
                    break
            
            # Kontroluje, zda počet členů báze nepřekročil počet trénovacích vzorků
            num_terms = len(self.active_multi_indices) + \
                        len(self.admissible_multi_indices) +\
                        len(new_admissible_multi_indices)
            #print("num_terms")
           # print("\033[95m" + str(num_terms) + "\033[0m")
            
            if num_terms >= len(self.pce.exp_design_inputs):
                if termination_info:
                    print("Adaptive basis construction terminated:" 
                        + " basis cardinality reached experimental design size.")
                break
             
            
            self.admissible_multi_indices += new_admissible_multi_indices  #Přidá nově nalezené přípustné multi-indexy do stávající množiny přípustných indexů
           #print(" self.admissible_multi_indices")
            #print("\033[95m" + str( self.admissible_multi_indices) + "\033[0m")
            all_multi_indices = self.active_multi_indices + self.admissible_multi_indices
           # print("all_multi_indices")
            #print("\033[95m" + str(all_multi_indices) + "\033[0m")                                                       
             #self.pce.set_multi_index_set(all_multi_indices) #Nastaví kompletní množinu indexů do PCE modelu
           # print("  self.pce.set_multi_index_set")
            #print("\033[95m" + str(  self.pce.set_multi_index_set) + "\033[0m")
             #self.pce.construct_basis()
            # print("  self.pce.construct_basis()")
             #print("\033[95m" + str(self.pce.construct_basis)+ "\033[0m")
            self.pce.compute_coefficients()
            #print(" self.pce.compute_coefficients")
           # print("\033[95m" + str( self.pce.compute_coefficients) + "\033[0m")
            
            #print(" konec")

            # Výběr a přesun nejvýznamnějšího indexu
            idx = len(self.active_multi_indices)
            #print("idx")
            #print("\033[95m" + str(idx) + "\033[0m")
            admissible_coefficients = self.pce.coefficients[idx:].tolist()
            #print("admissible_coefficients")
            #print("\033[95m" + str(admissible_coefficients) + "\033[0m")
            admissible_coefficients_array = np.array(admissible_coefficients)  # Convert to numpy array first
                # Kontrola dimenze pole (počet os)
            if len(admissible_coefficients_array.shape) == 1:
    # Pokud je pole 1D (např. [1, 2, 3]):
    
    # 1. np.abs() - převede všechny hodnoty na absolutní hodnoty
    # 2. np.sum() - sečte všechny absolutní hodnoty
    # Výsledek je jedno číslo (skalár) - celkový součet absolutních hodnot
                aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients))
    
            else:
    # Pokud je pole vícerozměrné (např. 2D jako [[1,2], [3,4]]):
    
    # 1. np.abs() - převede všechny hodnoty na absolutní hodnoty
    # 2. np.sum(axis=1) - sečte hodnoty po řádcích
    # Výsledek je pole součtů pro každý řádek (např. [3, 7] pro příklad výše)
                aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1)
            #print("aggregated_admissible_coefficients")
            #print("\033[95m" + str(aggregated_admissible_coefficients) + "\033[0m")
            
            help_index = np.argmax(aggregated_admissible_coefficients)
           #print("help_index")
            #print("\033[95m" + str(help_index) + "\033[0m")
            max_admissible_multi_index = self.admissible_multi_indices.pop(help_index)
            #print("max_admissible_multi_index")
            #print("\033[95m" + str(max_admissible_multi_index) + "\033[0m")
            self.active_multi_indices.append(max_admissible_multi_index)
            #print(" self.active_multi_indices.append")
            #print("\033[95m" + str(self.active_multi_indices.append) + "\033[0m")
            
    def construct_active_pce(self):      ## Zacit zde
        pce = PolynomialChaosExpansion(self.pdf, 
                            self.exp_design_in, 
                            self.exp_design_out)
        pce.set_multi_index_set(self.active_multi_indices)
        pce.construct_basis()
        pce.compute_coefficients()
        return pce
    
    # nový polynomiální chaosový expanzní (PCE) model, který kombinuje aktuální aktivní i přípustné multi-indexy
    # Není to jen "srovnání" – přímo ovlivňuje sestavu týmu!
    def construct_augmented_pce(self):
        pce = PolynomialChaosExpansion(self.pdf, 
                                    self.exp_design_in, 
                                    self.exp_design_out)
        pce.set_multi_index_set(self.active_multi_indices + 
                            self.admissible_multi_indices)
        pce.construct_basis()
        pce.compute_coefficients()
        return pce      

    # Vytvoří optimalizovanou verzi rozšířeného PCE modelu s kontrolou stability.
    def construct_reduced_augmented_pce(self, max_condition_number=1e2):
        # compute augmented pce
        pce = self.construct_augmented_pce()
        while True:
            # exit if the condition number is acceptable
            if pce.condition_number <= max_condition_number and\
                len(pce.multi_index_set) <= len(pce.exp_design_inputs):
                break
            # remove single- and multi-index with minimum contribution
            idx_min = np.argmin(np.sum(np.abs(pce.coefficients), axis=1))
            pce.multi_index_set.pop(idx_min)
            pce.single_index_set.pop(idx_min)
            # compute pce basis and coefficients with reduced multi-index set
            pce.construct_basis()
            pce.compute_coefficients()
        # re-order multi-indices and coefficients
        # coeffs = pce.coefficients
        # midx = np.array(pce.multi_index_set)
        # coeffs_aggr = np.sum(np.abs(coeffs), axis=1)
        # order_idx = np.flip(np.argsort(coeffs_aggr))
        # midx_ord = midx[order_idx, :].tolist()
        # pce.set_multi_index_set(midx_ord)
        # pce.construct_basis()
        # pce.compute_coefficients()
        return pce

In [120]:
import openturns as ot
import numpy as np
import time
from sklearn.metrics import mean_squared_error
from UQpy.distributions import Uniform, JointIndependent

def root_mean_squared_error(y_true, y_pred, multioutput='uniform_average'):
    mse = mean_squared_error(y_true, y_pred, multioutput=multioutput)
    if isinstance(mse, np.ndarray):
        return np.sqrt(mse)
    return np.sqrt(mse)

def test_SensitivityAdaptivePCE():
    # Nastavení parametrů
    dim = 3  # Počet vstupních proměnných
    size = 1000  # Počet trénovacích vzorků
    test_size = 50  # Počet testovacích vzorků
    
    # Vytvoření rovnoměrného rozdělení
    marginals = [Uniform(0, 1) for _ in range(dim)] 
    print(marginals)
    
    # distribution = ot.ComposedDistribution(marginals)
    distribution = JointIndependent(marginals=marginals)
    print(distribution)
    num_inputs = len(marginals)
    print("\033[95m" + str(distribution) + "\033[0m")
    
    # Generování trénovacích dat
    sampler = MonteCarloSampling(distributions=distribution, nsamples=size)  
    exp_design_in = sampler.samples
    
    # Testovací model s více výstupy
    def model(x):
        x_np = np.array(x)
        return np.column_stack((
            x_np[:, 0] + 2 * x_np[:, 1] + 3 * x_np[:, 2],
            x_np[:, 0] * x_np[:, 1] + x_np[:, 2],
            np.sin(x_np[:, 0]) + np.cos(x_np[:, 1]),
            np.exp(x_np[:, 2])
        ))
    
    exp_design_out = model(exp_design_in)
    
    # Generování testovacích dat
    test_experiment = MonteCarloSampling(distributions=distribution, nsamples=test_size)
    test_in = test_experiment.samples
    test_out = model(test_in)
    
    # Vytvoření a trénink adaptivního PCE modelu
    t0 = time.time()
    sapce = SensitivityAdaptivePCE(pdf=distribution, exp_design_in=exp_design_in, exp_design_out=exp_design_out, max_partial_degree=10,tolerance=1e-3, num_inputs=num_inputs)
    sapce.construct_adaptive_basis(max_condition_number=1e3)
    pce = sapce.pce  # Předpokládáme, že třída SensitivityAdaptivePCE má tento atribut
    print("pce")
    print("\033[95m" + str(pce) + "\033[0m")
    
    # Předpovědi na testovacích datech
    pce_predictions = pce.predict(test_in)
    print("pce_predictions")
    print("\033[95m" + str(pce_predictions) + "\033[0m")
    # Vyhodnocení přesnosti
    vector_rmse = root_mean_squared_error(test_out, pce_predictions, multioutput='raw_values')
    avg_rmse = root_mean_squared_error(test_out, pce_predictions, multioutput='uniform_average')
    
    # Výpočet R² skóre
    r2 = []
    for j in range(test_out.shape[1]):
        ss_res = np.sum((test_out[:, j] - pce_predictions[:, j])**2)
        ss_tot = np.sum((test_out[:, j] - np.mean(test_out[:, j]))**2)
        r2.append(1 - ss_res/ss_tot)

    mean = pce.compute_mean()
    stddev = pce.compute_stddev()

    print("Střední hodnota:", mean)
    print("Směrodatná odchylka:", stddev)
    
    # Výstupy modelu
    pce_mean = pce.compute_mean()
    print("pce_mean")
    print("\033[95m" + str(pce_mean) + "\033[0m")

    pce_variance = pce.compute_variance()
    print("pce_variance")
    print("\033[95m" + str(pce_variance) + "\033[0m")
   
    pce_std = np.sqrt(pce_variance)
    print("pce_std")
    print("\033[95m" + str(pce_std) + "\033[0m")
# Spuštění testu
# Spuštění testu
test_SensitivityAdaptivePCE()

[<UQpy.distributions.collection.Uniform.Uniform object at 0x000001A75A1C8050>, <UQpy.distributions.collection.Uniform.Uniform object at 0x000001A75A4AEBD0>, <UQpy.distributions.collection.Uniform.Uniform object at 0x000001A75A4AE120>]
Jsem tu v sapce
construct_adaptive_basis
Adaptive basis construction terminated: basis cardinality reached experimental design size.
pce
pce_predictions
[[1.22712825 0.29083197 0.99625403 1.32661189]
 [0.89518418 0.07965217 1.60653879 1.05303727]
 [3.51556436 0.78036361 0.95622895 1.47864091]
 [4.05238964 0.75626404 0.64371814 1.92387726]
 [1.40199068 0.32919883 1.38778807 1.36965425]
 [1.48688635 0.21134676 1.78382724 1.15844491]
 [3.04031318 0.79920881 1.38327624 1.95486335]
 [0.94951141 0.13426828 1.46004469 1.08515992]
 [3.04972229 0.83280347 1.40157687 2.14218377]
 [2.21287535 0.4337268  1.16052309 1.28097302]
 [4.40964419 1.08858144 1.02062385 2.35335921]
 [3.9491505  1.02076325 1.21827964 2.13148644]
 [3.09494187 0.97712033 1.4185451  1.10208445]
 